In [1]:
import numpy as np

In [2]:
np

<module 'numpy' from '/usr/local/lib/python3.8/dist-packages/numpy/__init__.py'>

In [3]:
a = np.array(range(10))

In [6]:
a[None:None]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])